In [1]:
import torch, pickle, time, os, random
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
# accelerate huggingface to GPU
if torch.cuda.is_available():
    from accelerate import Accelerator
    accelerator = Accelerator()
    device = accelerator.device
from pysr import pysr, best
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

torch.manual_seed(42)
random.seed(42)


print('Loading data')

# xs = pickle.load(open(osp.expanduser(f"~/../../../scratch/gpfs/cj1223/GraphStorage/standard_raw_final_6t/xs.pkl"), 'rb'))
# ys = pickle.load(open(osp.expanduser(f"~/../../../scratch/gpfs/cj1223/GraphStorage/standard_raw_final_6t/ys.pkl"), 'rb'))
# n_targ = len(ys[0])
# print('Loaded data')


Loading data


In [2]:
help(pysr)

Help on function pysr in module pysr.sr:

pysr(X, y, weights=None, **kwargs)



In [41]:
from torch.nn import ReLU, Linear, Module, LayerNorm, Sequential
class MLP(Module):
    def __init__(self, n_in=len(xs[0]), n_out = len(ys[0]), hidden=128, nlayers=6, layer_norm=True):
        super().__init__()
        layers = [Linear(n_in, hidden), ReLU()]
        for i in range(nlayers):
            layers.append(Linear(hidden, hidden))
            layers.append(ReLU()) 
        if layer_norm:
            layers.append(LayerNorm(hidden)) #yay
        layers.append(Linear(hidden, n_out))
        self.mlp = Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)

class HaloData(Dataset):
    def __init__(self, x, y, xcols = np.arange(34), ycols = np.arange(6)):
        self.x = x[:, xcols]
        self.y = y[:, ycols]
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [42]:
# data=data
batch_size=int(2**8) # 2^8 = 256
split=0.8

criterion = torch.nn.MSELoss()

l1_lambda = 1e-4
l2_lambda = 0
hidden = 128

train_data = HaloData(xs[:int(len(ys)*split)], ys[:int(len(ys)*split)])
test_data = HaloData(xs[int(len(ys)*split):], ys[int(len(ys)*split):])
train_loader = DataLoader(train_data, batch_size=512, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=512, shuffle=False)

yss, preds=[],[]
model = MLP(hidden=hidden)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
print('GPU ', next(model.parameters()).is_cuda)
# Initialize our train function
def train():
    model.train()
    for dat in train_loader: 
        x, y = dat
        out = model(x)  
        loss = criterion(out, y.view(-1,n_targ)) 
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())


        loss = loss + l1_lambda * l1_norm + l2_lambda * l2_norm
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad() 
#     print(loss, l1_norm*l1_lambda, l2_norm*l2_lambda)


##### test function #####

def test(loader): 
    model.eval()
    outs = []
    ys = []
    with torch.no_grad(): 
        for dat in loader: 
            x, y = dat
            out = model(x) 
            ys.append(y.view(-1,n_targ))
            outs.append(out)
    outss=torch.vstack(outs)
    yss=torch.vstack(ys)
    return torch.std(outss - yss, axis=0), outss, yss


GPU  False


In [43]:
n_epochs=200

tr_acc, te_acc = [], []
start=time.time()
for epoch in tqdm(range(n_epochs)):
#     print(epoch)
    train()
    if (epoch+1)%10==0:
        train_acc, _ , _ = test(train_loader)
        test_acc, _ , _ = test(test_loader)
        tr_acc.append(train_acc.cpu())
        te_acc.append(test_acc.cpu())
        print(f'Epoch: {epoch+1:03d}, Train scatter: {np.round(train_acc.cpu().numpy(), 4)} \n \
        Test scatter: {np.round(test_acc.cpu().numpy(), 4)}')
stop=time.time()
spent=stop-start
print(f"{spent:.2f} seconds spent training, {spent/n_epochs:.3f} seconds per epoch. Processed {len(ys)*split*n_epochs/spent:.0f} trees per second")



  5%|███                                                           | 10/200 [00:23<08:05,  2.56s/it]

Epoch: 010, Train scatter: [0.1638 0.2226 0.273  0.4054 0.4029 0.1949] 
         Test scatter: [0.156  0.2227 0.2627 0.3987 0.3962 0.1916]


 10%|██████▏                                                       | 20/200 [00:50<09:08,  3.05s/it]

Epoch: 020, Train scatter: [0.1637 0.2212 0.2699 0.4017 0.3995 0.1922] 
         Test scatter: [0.1558 0.2213 0.2594 0.3973 0.3948 0.1889]


 15%|█████████▎                                                    | 30/200 [01:14<07:22,  2.61s/it]

...

Epoch: 180, Train scatter: [0.1552 0.2122 0.2563 0.3988 0.3966 0.187 ] 
         Test scatter: [0.1473 0.213  0.245  0.3949 0.3926 0.184 ]


 95%|█████████████████████████████████████████████████████████▉   | 190/200 [07:34<00:31,  3.16s/it]

Epoch: 190, Train scatter: [0.1546 0.2109 0.2555 0.3982 0.3959 0.1861] 
         Test scatter: [0.1469 0.2116 0.2442 0.3933 0.391  0.183 ]


100%|█████████████████████████████████████████████████████████████| 200/200 [07:57<00:00,  2.39s/it]

Epoch: 200, Train scatter: [0.1551 0.2111 0.2557 0.3991 0.3968 0.1867] 
         Test scatter: [0.1477 0.2122 0.245  0.3955 0.3931 0.1835]
477.89 seconds spent training, 2.389 seconds per epoch. Processed 36370 trees per second


In [51]:
help(model.mlp[14])

Help on LayerNorm in module torch.nn.modules.normalization object:

class LayerNorm(torch.nn.modules.module.Module)
 |  LayerNorm(normalized_shape: Union[int, List[int], torch.Size], eps: float = 1e-05, elementwise_affine: bool = True, device=None, dtype=None) -> None
 |  
 |  Applies Layer Normalization over a mini-batch of inputs as described in
 |  the paper `Layer Normalization <https://arxiv.org/abs/1607.06450>`__
 |  
 |  .. math::
 |      y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta
 |  
 |  The mean and standard-deviation are calculated separately over the last
 |  certain number dimensions which have to be of the shape specified by
 |  :attr:`normalized_shape`.
 |  :math:`\gamma` and :math:`\beta` are learnable affine transform parameters of
 |  :attr:`normalized_shape` if :attr:`elementwise_affine` is ``True``.
 |  The standard-deviation is calculated via the biased estimator, equivalent to
 |  `torch.var(input, unbiased=False)`.
 |  
 |  

In [22]:
from datetime import date
today = date.today()

today = today.strftime("%d%m%y")

torch.save(model.state_dict(),f'trained_models/model_{epoch}_{today}.pt') 

In [24]:
load=False
if load:
    model = GCN(hidden_channelse=128)
    model.load_state_dict(torch.load(f'trained_models', f'model_{epoch}_{date}.pt'))

In [25]:
print('decoder', sum(p.abs().sum() for p in model.f.parameters())/sum(p.numel() for p in model.f.parameters())*100)

print('encoder', sum(p.abs().sum() for p in model.g1.parameters())/sum(p.numel() for p in model.g1.parameters())*100)
print('edge', sum(p.abs().sum() for p in model.g2.parameters())/sum(p.numel() for p in model.g2.parameters())*100)
print('both', sum(p.abs().sum() for p in model.g3.parameters())/sum(p.numel() for p in model.g3.parameters())*100)

decoder tensor(1.0244, grad_fn=<MulBackward0>)
encoder tensor(1.0797, grad_fn=<MulBackward0>)
edge tensor(0.5381, grad_fn=<MulBackward0>)
both tensor(1.1271, grad_fn=<MulBackward0>)


In [27]:
# model.load_state_dict(torch.load(osp.join(pointer, model_runs[k], 'trained_model', 'model.pt')))

In [84]:
pysr_loader=DataLoader(test_data[:50], batch_size=batch_size, shuffle=0, num_workers=4)    

y_f = []
y_g1 = []
y_g2 = []
y_g3 = []
x_g1 = []
y_t = []
batch = []
i=0
for dat in pysr_loader:
    print(i)
    x_g1.append(dat.x.cpu().detach().numpy())
    yg1=model.g1(dat.x)
    y_g1.append(yg1.cpu().detach().numpy())
    
    adj = dat.edge_index
    neighbours = yg1
    xg2 = scatter_add(neighbours[adj[0]], adj[1], dim=0)
    x_g2.append(xg2)
    yg2=model.g2(xg2)
    y_g2.append(yg2.cpu().detach().numpy())
    
    yg1[adj[1]]+=yg2[adj[1]]
    
    yg3 = model.g3(yg1)
    y_g3.append(yg3.cpu().detach().numpy())
    x_f = global_add_pool(yg3, dat.batch)
    
    yf = model.f(x_f)
    y_f.append(yf.cpu().detach().numpy())
    
    batch.append(dat.batch.cpu().detach().numpy())
    y_t.append(dat.y.cpu().detach().numpy())
    i+=1

0


In [85]:
N=25
vals, counts = np.unique(batch[0], return_counts=1)
l = np.cumsum(counts)[N-1]
x_g1_pysr=np.vstack(x_g1[0][:l])
y_g1_pysr=np.vstack(y_g1[0][:l])
y_g2_pysr=np.vstack(y_g2[0][:l])
y_g3_pysr=np.vstack(y_g3[0][:l])
y_f_pysr=np.vstack(y_f[0][:l])

b_pysr = batch[0][:l]

In [86]:
##
g1_equations = pysr(
    X=x_g1_pysr, y=y_g1_pysr,
    procs=4,
    niterations=20,
    populations=20,
    useFrequency=True,
    multithreading=True, 
    binary_operators=["plus", "sub", "mult", "div"],
    unary_operators = ['log10_abs', 'sqrt_abs', 'exp', 'log'], ##still need a general power law
    batching=1, 
    batchSize=256,
    maxsize=10, update=False
)

Running on julia -O3 --threads 4 /tmp/tmpyc0xp9yp/runfile.jl


  Activating environment at `~/.conda/envs/juptorch_julia/lib/python3.9/site-packages/Project.toml`


Started!

Cycles per second: 6.440e+03
Head worker occupation: 6.6%
Progress: 3 / 1200 total iterations (0.250%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.787e+00  7.562e-01  x1
3           1.111e+00  4.598e-01  (2.434756 * x1)
5           9.573e-01  7.446e-02  (-0.39906436 + (2.5171049 * x1))
7           7.673e-01  1.106e-01  (-0.2528663 + (x4 + (2.1287289 * x1)))
8           7.581e-01  1.199e-02  ((2.0636811 * x1) + log_abs(0.94439185 + x4))
9           6.789e-01  1.104e-01  (x1 + ((-0.57546425 * x0) + (3.2942095 * x2)))

Best equations for output 3
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           9.371e+00  2.264e-01  x1
2           9.097e+00  2.968e-02  exp(x1)
3           6.752e+00  2.981e-01  (2.793


Cycles per second: 3.380e+03
Head worker occupation: 0.9%
Progress: 13 / 1200 total iterations (1.083%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           9.421e-01  4.605e-01  x1
3           7.979e-01  8.308e-02  (0.3797928 + x1)
4           6.190e-01  2.539e-01  (x1 - log10_abs(x2))
6           6.178e-01  9.897e-04  (x1 - (0.009313339 + log10_abs(x2)))
7           6.145e-01  5.290e-03  (x1 - (log_abs(sqrt_abs(x2)) - -0.10132865))
8           5.698e-01  7.561e-02  (1.8673731 + (x1 + (-2.3141558 * sqrt_abs(x2))))
10          5.679e-01  1.624e-03  (x1 + (-1.0557032 * log10_abs(x2 + (-0.1134993 * x3))))

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.787e+00  7.562e-01  x1
3           1.088e+00  4.701e-01  (4.234271 * x2)
5           8.302e-01  1.354e-01  (x4 - (x1 / -0.49075502))
7           7.658e-01  4.035e


Cycles per second: 2.680e+03
Head worker occupation: 0.5%
Progress: 21 / 1200 total iterations (1.750%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           9.421e-01  4.605e-01  x1
3           7.990e-01  8.240e-02  (x1 + 0.34660563)
4           6.190e-01  2.553e-01  (x1 - log10_abs(x2))
7           5.908e-01  1.554e-02  ((2.9686167 * x1) + (-3.5632546 * x2))
8           5.679e-01  3.962e-02  ((2.3849034 - log_abs(x1)) * (x1 - x2))
10          5.659e-01  1.701e-03  (x1 + (-1.07813 * log10_abs(x2 + (-0.09936239 * x3))))

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.787e+00  7.562e-01  x1
3           1.111e+00  4.598e-01  (2.434756 * x1)
5           8.290e-01  1.464e-01  (x4 + (2.076545 * x1))
7           7.673e-01  3.867e-02  (-0.25267908 + (x4 + (2.1288564 * x1)))
9           6.693e-01  6.832e-02  (-0.544885


Cycles per second: 2.770e+03
Head worker occupation: 0.7%
Progress: 30 / 1200 total iterations (2.500%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           9.421e-01  4.605e-01  x1
3           7.979e-01  8.308e-02  (0.37972298 + x1)
4           6.190e-01  2.539e-01  (x1 - log10_abs(x2))
6           6.140e-01  4.015e-03  (x1 + log10_abs(0.85042274 / x2))
8           6.078e-01  5.104e-03  (x1 + log10_abs(-0.3060487 + (0.8593486 / x2)))
9           5.971e-01  1.785e-02  ((x1 - log10_abs(x2 * sqrt_abs(x1))) - 0.18158987)
10          5.677e-01  5.047e-02  (x1 + log10_abs((-1.0497826 / x2) + (-0.7018149 * x3)))

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.787e+00  7.562e-01  x1
3           1.232e+00  4.080e-01  (x1 / 0.48811185)
5           8.337e-01  1.954e-01  ((x1 + x4) + x1)
8           8.279e-01  2.349e-03 

Killing process... will return when done.


In [87]:
eq = g1_equations[0].sort_values(by='score', ascending=False).iloc[0]
a=eq['lambda_format'](x_g1[0])
eq

Complexity                                                4
MSE                                                0.618991
score                                              0.253895
Equation                               (x1 - log10_abs(x2))
sympy_format                      x1 - log(Abs(x2))/log(10)
lambda_format    PySRFunction(X=>x1 - log(Abs(x2))/log(10))
Name: 2, dtype: object

In [88]:
eq = g1_equations[1].sort_values(by='score', ascending=False).iloc[0]
b=eq['lambda_format'](x_g1[0])
eq

Complexity                                          3
MSE                                          1.143048
score                                        0.445626
Equation                            (x1 / 0.37977126)
sympy_format                      2.63316397349289*x1
lambda_format    PySRFunction(X=>2.63316397349289*x1)
Name: 1, dtype: object

In [89]:
eq = g1_equations[2].sort_values(by='score', ascending=False).iloc[0]
c=eq['lambda_format'](x_g1[0])
eq

Complexity                                   3
MSE                                   6.751929
score                                 0.298143
Equation                      (2.7937005 * x1)
sympy_format                      2.7937005*x1
lambda_format    PySRFunction(X=>2.7937005*x1)
Name: 2, dtype: object

In [90]:
g1_an_pysr = np.vstack([a,b,c])

In [ ]:
g2_equations = pysr(
    X=y_g1_pysr, y=y_g2_pysr,
    procs=4,
    niterations=20,
    populations=20,
    useFrequency=True,
    multithreading=True, 
    binary_operators=["plus", "sub", "mult", "div", 'pow'],
    unary_operators = ['log10_abs', 'sqrt_abs', 'exp', 'log'], ##still need a general power law
    batching=1, 
    batchSize=256,
    maxsize=10, update=False
)

Running on julia -O3 --threads 4 /tmp/tmpgyms7yzv/runfile.jl


  Activating environment at `~/.conda/envs/juptorch_julia/lib/python3.9/site-packages/Project.toml`


Started!

Cycles per second: 5.180e+03
Head worker occupation: 8.3%
Progress: 2 / 1200 total iterations (0.167%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.068e-02  -0.000e+00  -0.23073237
5           1.325e-02  1.114e-01  (-0.23157844 + (0.035398062 * x1))

Best equations for output 3
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           1.021e+00  -0.000e+00  1.2790511
4           8.282e-01  6.981e-02  sqrt_abs(1.3321475 - x0)
5           5.994e-01  3.233e-01  (1.2852532 + (-0.26651594 * x1))
6           5.982e-01  2.089e-03  sqrt_abs(1.9554164 + (-0.6193561 * x1))

Press 'q' and then <enter> to stop execution early.

Cycles per second: 4.950e+03
Head worker occupation: 5.1%
Progress: 3 / 1200 total iteration


Cycles per second: 3.420e+03
Head worker occupation: 0.7%
Progress: 10 / 1200 total iterations (0.833%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.406e-02  -0.000e+00  -0.3479579
5           1.762e-02  7.788e-02  (-0.3471601 + (-0.032939747 * x1))
8           1.757e-02  9.525e-04  (-0.7191795 + exp(-1.0134043 + (-0.092179745 * x1)))
9           1.641e-02  6.816e-02  (-0.362665 + ((-0.045649357 * x1) + (0.013580261 * x2)))

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.068e-02  -0.000e+00  -0.23073237
5           1.325e-02  1.114e-01  (-0.23157844 + (0.035398062 * x1))

Best equations for output 3
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           1.021e+00  1.192e-07  1.278901
4           6.409e-01  1.553e-01  exp(-0.25171164 * x1)
5      


Cycles per second: 5.100e+03
Head worker occupation: 0.5%
Progress: 22 / 1200 total iterations (1.833%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.406e-02  -0.000e+00  -0.3479579
5           1.762e-02  7.788e-02  (-0.3471601 + (-0.032939747 * x1))
8           1.757e-02  9.525e-04  (-0.7191795 + exp(-1.0134043 + (-0.092179745 * x1)))
9           1.641e-02  6.816e-02  (-0.362665 + ((-0.045649357 * x1) + (0.013580261 * x2)))

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.068e-02  -0.000e+00  -0.23073237
5           1.325e-02  1.114e-01  (-0.23157844 + (0.035398062 * x1))

Best equations for output 3
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           1.021e+00  1.192e-07  1.278901
4           6.409e-01  1.553e-01  exp(-0.25171337 * x1)
5      


Cycles per second: 4.950e+03
Head worker occupation: 0.4%
Progress: 35 / 1200 total iterations (2.917%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.406e-02  -0.000e+00  -0.3479579
5           1.762e-02  7.788e-02  (-0.3471601 + (-0.032939747 * x1))
8           1.757e-02  9.525e-04  (-0.7191795 + exp(-1.0134043 + (-0.092179745 * x1)))
9           1.641e-02  6.816e-02  (-0.362665 + ((-0.045649357 * x1) + (0.013580261 * x2)))

Best equations for output 2
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.068e-02  -0.000e+00  -0.23073237
5           1.325e-02  1.114e-01  (-0.23157844 + (0.035398062 * x1))

Best equations for output 3
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           1.021e+00  1.192e-07  1.278901
4           6.409e-01  1.553e-01  exp(-0.2517181 * x1)
5       